# TNT Multi-Species: Flow Curve Fitting

**Objectives:**
- Fit TNT multi-species model to steady shear flow curve data
- Understand multiple independent bond species with different lifetimes
- Decompose total stress into per-species contributions
- Analyze discrete relaxation spectrum
- Compare NLSQ and Bayesian inference

## Setup

In [1]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTMultiSpecies

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_ml_ikh_flow_curve,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_multi_species_param_names,
    plot_multi_species_spectrum,
    plot_mode_decomposition,
)

from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

## Theory: Multiple Independent Bond Species

The TNT multi-species model describes materials with multiple independent bond populations, each with its own modulus $G_i$ and lifetime $\tau_{b,i}$.

**Total stress:**
$$\sigma = \sum_{i=0}^{N-1} \sigma_i + \eta_s \dot{\gamma}$$

**Per-species contribution (steady shear):**
$$\sigma_i = \frac{G_i \tau_{b,i} \dot{\gamma}}{1 + (\tau_{b,i} \dot{\gamma})^2}$$

**Key physics:**
- Fast bonds (short $\tau_{b,0}$): Dominate high shear rates
- Slow bonds (long $\tau_{b,1}$): Dominate low shear rates
- Discrete relaxation spectrum with 2 modes
- Stress superposition from independent populations

**Parameters (2 species):**
- $G_0$: Fast species modulus [Pa]
- $\tau_{b,0}$: Fast species bond lifetime [s]
- $G_1$: Slow species modulus [Pa]
- $\tau_{b,1}$: Slow species bond lifetime [s]
- $\eta_s$: Solvent viscosity [Pa·s]

## Load Data

In [2]:
gamma_dot, sigma = load_ml_ikh_flow_curve("ARES_up")

print(f"Data points: {len(gamma_dot)}")
print(f"Shear rate range: {gamma_dot.min():.4f} to {gamma_dot.max():.2f} s^-1")
print(f"Stress range: {sigma.min():.2f} to {sigma.max():.2f} Pa")

fig, ax = plt.subplots(figsize=(8, 6))
ax.loglog(gamma_dot, sigma, 'o', label='Data', markersize=8)
ax.set_xlabel(r'Shear Rate $\dot{\gamma}$ [s$^{-1}$]', fontsize=12)
ax.set_ylabel(r'Stress $\sigma$ [Pa]', fontsize=12)
ax.set_title('Carbopol Flow Curve', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.close("all")
plt.close('all')

Data points: 21
Shear rate range: 0.0100 to 100.00 s^-1
Stress range: 0.75 to 51.46 Pa


## NLSQ Fitting

In [3]:
model = TNTMultiSpecies(n_species=2)
param_names = get_tnt_multi_species_param_names(n_species=2)
print(f"Parameters: {param_names}")

start_time = time.time()
model.fit(gamma_dot, sigma, test_mode="flow_curve", method="scipy")
nlsq_time = time.time() - start_time

print(f"\nNLSQ fit completed in {nlsq_time:.2f} s")
print("\nFitted parameters:")
for name in param_names:
    print(f"  {name}: {model.parameters.get_value(name):.6e}")

Parameters: ['G_0', 'tau_b_0', 'G_1', 'tau_b_1', 'eta_s']



NLSQ fit completed in 0.43 s

Fitted parameters:
  G_0: 1.876510e+01
  tau_b_0: 3.560245e-02
  G_1: 1.876513e+01
  tau_b_1: 3.561114e-03
  eta_s: 3.560150e-12


In [4]:
# Compute metrics for plot title
metrics = compute_fit_quality(sigma, model.predict(gamma_dot, test_mode='flow_curve'))

# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    gamma_dot, sigma, model, test_mode="flow_curve",
    param_names=param_names, log_scale=True,
    xlabel=r'Shear rate $\dot{\gamma}$ (s$^{-1}$)',
    ylabel=r'Shear stress $\sigma$ (Pa)',
    title=f'NLSQ Fit (R² = {metrics["R2"]:.4f})'
)
plt.close("all")

## Physical Analysis: Per-Species Stress Decomposition

In [5]:
# Create prediction grid
gamma_dot_pred = np.logspace(np.log10(gamma_dot.min()), np.log10(gamma_dot.max()), 200)

fig = plot_mode_decomposition(model, gamma_dot_pred, "flow_curve")
plt.close("all")
plt.close('all')

print("\nSpecies characteristics:")
G_0 = model.parameters.get_value('G_0')
tau_b_0 = model.parameters.get_value('tau_b_0')
G_1 = model.parameters.get_value('G_1')
tau_b_1 = model.parameters.get_value('tau_b_1')

print(f"\nSpecies 0 (fast):")
print(f"  G_0 = {G_0:.3e} Pa")
print(f"  tau_b_0 = {tau_b_0:.3e} s")
print(f"  Crossover rate ~ 1/tau_b_0 = {1/tau_b_0:.3e} s^-1")

print(f"\nSpecies 1 (slow):")
print(f"  G_1 = {G_1:.3e} Pa")
print(f"  tau_b_1 = {tau_b_1:.3e} s")
print(f"  Crossover rate ~ 1/tau_b_1 = {1/tau_b_1:.3e} s^-1")

print(f"\nLifetime ratio: tau_b_1/tau_b_0 = {tau_b_1/tau_b_0:.2f}")
print(f"Modulus ratio: G_1/G_0 = {G_1/G_0:.2f}")


Species characteristics:

Species 0 (fast):
  G_0 = 1.877e+01 Pa
  tau_b_0 = 3.560e-02 s
  Crossover rate ~ 1/tau_b_0 = 2.809e+01 s^-1

Species 1 (slow):
  G_1 = 1.877e+01 Pa
  tau_b_1 = 3.561e-03 s
  Crossover rate ~ 1/tau_b_1 = 2.808e+02 s^-1

Lifetime ratio: tau_b_1/tau_b_0 = 0.10
Modulus ratio: G_1/G_0 = 1.00


## Discrete Relaxation Spectrum

In [6]:
fig = plot_multi_species_spectrum(model)
plt.close("all")
plt.close('all')

print("\nSpectrum interpretation:")
print("- Two discrete relaxation times from two bond species")
print("- Fast species contributes at short times/high frequencies")
print("- Slow species contributes at long times/low frequencies")
print("- No continuous distribution (discrete modes only)")


Spectrum interpretation:
- Two discrete relaxation times from two bond species
- Fast species contributes at short times/high frequencies
- Slow species contributes at long times/low frequencies
- No continuous distribution (discrete modes only)


## Important Note on Model Applicability

**Carbopol is not a multi-species network.** It is a single cross-linked polymer gel with a continuous relaxation spectrum. This notebook demonstrates:
1. The multi-species workflow
2. Species decomposition analysis tools
3. How to interpret discrete spectra

Multi-species models are appropriate for:
- Binary blends (two polymer types)
- Multi-length-scale networks
- Materials with distinct fast/slow relaxation mechanisms

For Carbopol, single-species TNT models are more physically appropriate.

## Bayesian Inference

In [7]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    result_bayes = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    result_bayes = model.fit_bayesian(
        gamma_dot, sigma,
        test_mode='flow_curve',
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)
To run Bayesian analysis, run with FAST_MODE=0


## Convergence Diagnostics

In [8]:
# Skip convergence diagnostics in CI mode
if not FAST_MODE:
    print_convergence_summary(result_bayes, param_names)
else:
    print("FAST_MODE: Skipping convergence diagnostics")


FAST_MODE: Skipping convergence diagnostics


## ArviZ Diagnostics

In [9]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(result_bayes, 'to_inference_data'):
    display_arviz_diagnostics(result_bayes, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

FAST_MODE: Skipping ArviZ diagnostics


## NLSQ vs Bayesian Parameter Comparison

In [10]:
print_parameter_comparison(model, result_bayes.posterior_samples, param_names)


Parameter Comparison: NLSQ vs Bayesian
      Parameter          NLSQ        Median                          95% CI
---------------------------------------------------------------------------
            G_0         18.77         18.77  [18.77, 18.77]
        tau_b_0        0.0356        0.0356  [0.0356, 0.0356]
            G_1         18.77         18.77  [18.77, 18.77]
        tau_b_1      0.003561      0.003561  [0.003561, 0.003561]
          eta_s      3.56e-12      3.56e-12  [3.56e-12, 3.56e-12]


## Posterior Predictive: Flow Curve

In [11]:
# Posterior predictive check
if not FAST_MODE and hasattr(result_bayes, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        gamma_dot, sigma, model, result_bayes,
        test_mode="flow_curve", param_names=param_names,
        log_scale=True,
        xlabel=r'Shear rate $\\dot{\\gamma}$ (s$^{-1}$)',
        ylabel=r'Shear stress $\\sigma$ (Pa)'
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

FAST_MODE: Skipping posterior predictive


## Physical Interpretation

**Multi-species stress contributions:**
- Fast species dominates at high shear rates (short timescale)
- Slow species dominates at low shear rates (long timescale)
- Crossover region shows mixed contributions

**Discrete spectrum:**
- Two well-separated relaxation times
- No intermediate modes (discrete, not continuous)
- Lifetime ratio indicates timescale separation

**Uncertainty quantification:**
- Bayesian posterior captures parameter correlations
- Predictive uncertainty from finite data
- Species parameters may trade off in fitting

## Save Results

In [12]:
save_tnt_results(model, result_bayes, "multi_species", "flow_curve", param_names)
print("Results saved successfully.")

Results saved to /Users/b80985/Projects/rheojax/examples/tnt/../utils/../outputs/tnt/multi_species/flow_curve/
  nlsq_params_flow_curve.json: 5 parameters
  posterior_flow_curve.json: 500 draws
Results saved successfully.


## Key Takeaways

1. **Multi-species framework**: Independent bond populations with different lifetimes
2. **Stress superposition**: Total stress is sum of species contributions
3. **Timescale separation**: Fast/slow species dominate different shear rate regimes
4. **Discrete spectrum**: Finite number of relaxation modes
5. **Workflow demonstration**: This notebook shows analysis tools, not physical appropriateness for Carbopol
6. **Bayesian inference**: Quantifies parameter uncertainty and correlations
7. **Species decomposition**: Visualization tools reveal individual contributions